In [2]:
import torch
from transformers import DistilBertTokenizer, DistilBertForMultipleChoice

/home/ppxscal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Initialize the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForMultipleChoice.from_pretrained('distilbert-base-uncased')

# Define the soft prompt
L = 8  # Length of the soft prompt
d = model.distilbert.embeddings.word_embeddings.embedding_dim  # Dimension of the embeddings
soft_prompt = torch.nn.Parameter(torch.randn(L, d))
soft_prompt = soft_prompt.unsqueeze(1).repeat(1, 2, 1).unsqueeze(2).repeat(1,1,6,1)


# Encode a sample multiple choice question
choices = ["Hello, world!", "Goodbye, world!"]
encoded_choices = [tokenizer.encode(s, add_special_tokens=True) for s in choices]
input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2

# Get the input embeddings
input_embeddings = model.distilbert.embeddings.word_embeddings(input_ids)

print(f'input embeddings shape: {input_embeddings.shape}, soft prompt shape: {soft_prompt.shape}')

# Prepend the soft prompt to the input embeddings
combined_embeddings = torch.cat([soft_prompt, input_embeddings], dim=0)

# Pass the combined embeddings through the model
outputs = model(inputs_embeds=combined_embeddings)

Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


input embeddings shape: torch.Size([1, 2, 6, 768]), soft prompt shape: torch.Size([8, 2, 6, 768])


In [26]:
# Import necessary libraries
from transformers import DistilBertTokenizer, DistilBertForMultipleChoice
import torch
from datasets import load_dataset

# Initialize the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForMultipleChoice.from_pretrained('distilbert-base-uncased')

# Load the dataset
dataset = load_dataset('bigscience/P3', 'cos_e_v1.11_aligned_with_common_sense')

# Define the soft prompt
L = 8  # Length of the soft prompt
d = model.distilbert.embeddings.word_embeddings.embedding_dim  # Dimension of the embeddings
soft_prompt = torch.nn.Parameter(torch.randn(L, d))

# Let's assume we're working with the first example in the dataset
example = dataset['train'][1]
bro = example["inputs_pretokenized"]
moment = example["targets_pretokenized"]

print(f'Example: {bro}')
print(f'Answer: {moment}')
print(f'Dict: {example}')
# Tokenize the example
input_ids = tokenizer.encode(example['text'], return_tensors='pt')

# Get the input embeddings
input_embeddings = model.distilbert.embeddings.word_embeddings(input_ids)

# Prepend the soft prompt to the input embeddings
soft_prompt = soft_prompt.unsqueeze(0).repeat(input_embeddings.size(0), 1, 1)  # Adjust the soft prompt to match the batch size
combined_embeddings = torch.cat([soft_prompt, input_embeddings], dim=1)

# Pass the combined embeddings through the model
outputs = model(inputs_embeds=combined_embeddings)

Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Example: Here's a question and a few possible answers: 

Q: A John is a bum.  Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?
Possible A: bus depot, beach, train station, bridge, bridge

Why is "bridge" an answer aligned with human common sense? 

Answer: 
bums are well known to take up residence under bridges.
Dict: {'inputs': [947, 31, 7, 3, 9, 822, 11, 3, 9, 360, 487, 4269, 10, 1593, 10, 71, 1079, 19, 3, 9, 8524, 51, 5, 8718, 114, 8, 26524, 6, 3, 88, 1342, 1084, 48, 1843, 13, 5127, 3620, 5, 2840, 405, 3, 88, 619, 58, 29403, 71, 10, 2601, 14089, 6, 2608, 6, 2412, 2478, 6, 4716, 6, 4716, 1615, 19, 96, 9818, 121, 46, 1525, 7901, 15, 26, 28, 936, 1017, 1254, 58], 'inputs_pretokenized': 'Here\'s a question and a few possible answers: \n\nQ: A John is a bum.  Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?\nPossible A: bus depot, beach, train station, bridge, bridge\n\nWhy is "b

KeyError: 'text'